In [1]:
 from google.colab import drive
drive.mount('/content/drive',force_remount=True)
DATA_PATH = "/content/drive/My Drive/Intro to AI Project/data/sample-5/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple

     |████████████████████████████████| 645kB 8.2MB/s 


In [3]:
import pickle
import json
import os
import numpy as np
from tqdm import *
import gpt_2_simple as gpt2
from datetime import datetime
import tensorflow as tf
import shutil

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
!nvidia-smi

Thu Dec  5 02:28:41 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
def process_data():
    data = pickle.load(open(DATA_PATH + 'recipe1m_train.pkl', 'rb'))
    recipe_file = open(DATA_PATH + "recipe.txt","w")
    for recipe in tqdm(data):
        recipe_file.write("<|startoftext|>")# recipe
        recipe_file.write("<soi>") # ingredients
        recipe_file.write(",".join(recipe["ingredients"]).replace("_"," "))
        recipe_file.write("<eoi>\n")
        recipe_file.write("<sot>") # title
        recipe_file.write(recipe["title"])
        recipe_file.write("<eot>\n")
        recipe_file.write("<soc>-") # content (instructions)
        recipe_file.write("\n-".join(recipe["instructions"]))
        recipe_file.write("<eoc>")
        recipe_file.write("<|endoftext|>\n")
    recipe_file.close()
    # save file to google drive
    file_name = shutil.copyfile(DATA_PATH + "recipe.txt","recipe.txt")

In [0]:
file_name = "recipe.txt"
model_name = '355M'
checkpoint = "/content/drive/My Drive/Intro to AI Project/model/sample5-355M/checkpoint"

In [0]:
gpt2.download_gpt2(model_name=model_name)

Fetching checkpoint: 1.05Mit [00:00, 444Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 108Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 774Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:08, 165Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 435Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 116Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 216Mit/s]                                                       


In [0]:
process_data()
sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              dataset=file_name,
              model_name=model_name,
              steps=10000,
              restore_from='fresh',
              checkpoint_dir=checkpoint,
              run_name='run0',
              print_every=1,
              sample_every=200,
              save_every=200
              )


100%|██████████| 127948/127948 [00:00<00:00, 141300.80it/s]


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use tensorflow.python.ops.op_selector.get_backward_walk_ops.
Loading checkpoint models/355M/model.ckpt
INFO:tensorflow:Restoring parameters from models/355M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [02:12<00:00, 132.89s/it]


dataset has 27930475 tokens
Training...
[1 | 10.90] loss=2.48 avg=2.48
[2 | 11.80] loss=2.48 avg=2.48
[3 | 12.69] loss=2.06 avg=2.34
[4 | 13.57] loss=2.37 avg=2.35
[5 | 14.47] loss=1.90 avg=2.26
[6 | 15.35] loss=2.03 avg=2.22
[7 | 16.25] loss=2.10 avg=2.20
[8 | 17.13] loss=2.91 avg=2.29
[9 | 18.03] loss=1.80 avg=2.24
[10 | 18.91] loss=1.95 avg=2.21
[11 | 19.81] loss=2.27 avg=2.21
[12 | 20.69] loss=2.29 avg=2.22
[13 | 21.58] loss=2.03 avg=2.20
[14 | 22.46] loss=1.76 avg=2.17
[15 | 23.34] loss=1.90 avg=2.15
[16 | 24.23] loss=1.86 avg=2.13
[17 | 25.13] loss=2.03 avg=2.12
[18 | 26.02] loss=1.95 avg=2.11
[19 | 26.91] loss=1.82 avg=2.10
[20 | 27.80] loss=2.15 avg=2.10
[21 | 28.68] loss=2.09 avg=2.10
[22 | 29.57] loss=1.95 avg=2.09
[23 | 30.45] loss=1.84 avg=2.08
[24 | 31.34] loss=1.76 avg=2.06
[25 | 32.23] loss=1.95 avg=2.06
[26 | 33.11] loss=1.69 avg=2.04
[27 | 34.01] loss=1.71 avg=2.03
[28 | 34.89] loss=1.79 avg=2.02
[29 | 35.77] loss=1.90 avg=2.01
[30 | 36.66] loss=1.74 avg=2.00
[31 | 37.

In [6]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run0',checkpoint_dir=checkpoint)

Loading checkpoint /content/drive/My Drive/Intro to AI Project/model/sample5-355M/checkpoint/run0/model-10000
INFO:tensorflow:Restoring parameters from /content/drive/My Drive/Intro to AI Project/model/sample5-355M/checkpoint/run0/model-10000


In [0]:
# val 999
gpt2.generate(sess,
              run_name='run0',
              length=250,
              temperature=1.0,
              prefix="<|startoftext|><soi>all purpose flour,whole wheat flour,baking soda,salt,quick cooking oat,all bran cereal,raisin bran cereal,grape nut cereal,raisin,unsalted butter,buttermilk,granulated sugar,brown sugar,egg<eoi>",
              truncate="<|endoftext|>",
              checkpoint_dir=checkpoint,
              nsamples=5,
              batch_size=5
              )

<|startoftext|><soi>all purpose flour,whole wheat flour,baking soda,salt,quick cooking oat,all bran cereal,raisin bran cereal,grape nut cereal,raisin,unsalted butter,buttermilk,granulated sugar,brown sugar,egg<eoi>
<sot>sunstruck bran muffins<eot>
<soc>-preheat oven to 350f line 12 muffin with paper or nonstick cooking cups.
-in 1 large bowl, whisk together whole wheat flour, whole wheat flour, soda, salt.
-in another large bowl, combine bran brand yolks and cereal, and stir with 1 whisk just until cereal flakes easily to small crumbs, about 1 minute.
-add raisins with 1 pinch of salt, stirring just until cereal flakes easily when it is added add about 67.61 l of cheese at 1 time, nearly mixing all of it until it is melted total 6.17 l .
-add more bran brand nuts, if desired.
-in 1 medium bowl, combine rasins with about 67.61 l of butter and stir to thoroughly combine.
-add rasins all broth and stir just until bread crumbs form. stir in granulated sugar and brown sugar, smooth batter w

In [0]:
#val 104
gpt2.generate(sess,
              run_name='run0',
              length=250,
              temperature=1.0,
              prefix="<|startoftext|><soi>egg,light brown sugar,buttermilk,water,brewed coffee,all purpose white flour,baking powder,baking soda,salt,ground clove,ground cinnamon,ground ginger,ground nutmeg,butter<eoi>",
              truncate="<|endoftext|>",
              checkpoint_dir=checkpoint,
              nsamples=5,
              batch_size=5
              )

<|startoftext|><soi>egg,light brown sugar,buttermilk,water,brewed coffee,all purpose white flour,baking powder,baking soda,salt,ground clove,ground cinnamon,ground ginger,ground nutmeg,butter<eoi>
<sot>hawaiian cheese bread<eot>
<soc>-preheat oven to 350 degrees f 175 degrees .
-grease 1 9x5 inch loaf pan.
-in 1 large bowl, stir together the dark chocolate chips, 2.82 l confectioners' sugar, the buttermilk, and the water.
-using 1 wire whisk, set the bowl aside.
-in 1 medium saucepan, stir together the coffee beans with 2.11 l of the water.
-bring the mixture to 1 boil over medium heat, and cook until the coffee beans are hot about 4 minutes .
-set the bread in the center of the dish.
-cool in the pan.
-to the bowl with the milk mixture, stir the powdered sugar, baking powder, baking soda, cloves, cinnamon, ginger, nutmeg, and 3.17 l of the butter until blended.
-cover the mixture tightly with 1 large sheet of plastic wrap, and place the loaf in the refrigerator until firm, 24 hours or

In [0]:
#val 16
gpt2.generate(sess,
              run_name='run0',
              length=1024,
              temperature=0.8,
              prefix="<|startoftext|><soi>chicken wing,cilantro,garlic clove,jalapeno,fish sauce,fresh lime juice,unsalted butter,flour,canola oil,kosher salt,black pepper<eoi>",
              truncate="<|endoftext|>",
              checkpoint_dir=checkpoint,
              nsamples=5,
              batch_size=5
              )

<|startoftext|><soi>chicken wing,cilantro,garlic clove,jalapeno,fish sauce,fresh lime juice,unsalted butter,flour,canola oil,kosher salt,black pepper<eoi>
<sot>seafood bbq chicken wings<eot>
<soc>-combine the cilantro, garlic, jalapeno, and fish sauce in 1 small bowl.
-place the wings in 1 medium bowl and pour in the marinade.
-marinate in the refrigerator at least 2 hours.
-heat the oil in 1 large nonstick skillet over medium heat.
-sprinkle the wings with salt and pepper.
-when just cooked, remove them using tongs and transfer to 1 cutting board.
-slice the wings lengthwise and onto alternating sides several times.
-cut across into 6 pieces.
-add some of the chopped wings to the marinade and keep warm until serving.
-repeat with the remaining wings and marinade, serve with the crab soup on side.<eoc>
<|startoftext|><soi>chicken wing,cilantro,garlic clove,jalapeno,fish sauce,fresh lime juice,unsalted butter,flour,canola oil,kosher salt,black pepper<eoi>
<sot>chicken wings with yellow 

In [15]:
gpt2.generate(sess,
              run_name='run0',
              length=1024,
              temperature=0.7,
              prefix="<|startoftext|><soi>chicken,carrot,egg,onion,pepper,oil,sugar,salt,flour<eoi>",
              truncate="<|endoftext|>",
              checkpoint_dir=checkpoint,
              nsamples=5,
              batch_size=5
              )

<|startoftext|><soi>chicken,carrot,egg,onion,pepper,oil,sugar,salt,flour<eoi>
<sot>slow cooker chicken<eot>
<soc>-combine flour, sugar, salt and pepper.
-add chicken and carrots.
-cook on low heat for 8 10 hours or until chicken is done.
-if oven proof, place chicken on rack.
-cook in oven at 400 degrees for 30 minutes.
-serve with rice or pilaf.
-enjoy.<eoc>
<|startoftext|><soi>chicken,carrot,egg,onion,pepper,oil,sugar,salt,flour<eoi>
<sot>chicken soup<eot>
<soc>-in 1 soup pot cover the chicken with the carrots and pork.
-add the onion, pepper, and sugar.
-cook over medium high heat for 3 minutes then add the eggs.
-cook, stirring constantly until the eggs are just set.<eoc>
<|startoftext|><soi>chicken,carrot,egg,onion,pepper,oil,sugar,salt,flour<eoi>
<sot>chicken and carrot soup<eot>
<soc>-grind the carrot in 1 pan and add water.
-add the onion and pepper.
-saute for 1 couple of minutes.
-add the carrots.
-add the chicken and eggs.
-add the flour and stir occasionally.
-simmer for 20

In [7]:
gpt2.generate(sess,
              run_name='run0',
              length=1024,
              temperature=0.8,
              prefix="<|startoftext|><soi>chicken,carrot,egg,onion,pepper,oil,sugar,salt,flour<eoi>",
              truncate="<|endoftext|>",
              checkpoint_dir=checkpoint,
              nsamples=5,
              batch_size=5
              )

<|startoftext|><soi>potato,bean,carrot,pepper,onion,salt,oil<eoi>
<sot>corn and potato soup<eot>
<soc>-boil the corn and put in 1 stock pot.
-add the potatoes and carrots, and simmer until vegetables are very tender about 10 minutes .
-add the beans, onion, salt and pepper, and mince the pepper.
-cover and cook for 10 minutes.
-meanwhile boil the onion with the pepper in 1 4.11 l saucepan until tender.
-add to the stock pot.
-cover and simmer for 10 minutes.
-stir in about one and one and one quarter of the water and bring to 1 boil.
-reduce heat and allow to cook for about 3 minutes.
-serve with 1 pat of butter for dipping.
-note. to make 1 lighter vegetable soup, simply brown the onions, carrots, and potatoes in the butter, not oil.
-add the peas, 1 little water, and 1 good pinch of salt and pepper to the mixture and boil until the peas are tender, 10 minutes.
-add more water if the mixture is too thick.
-just before serving, add the peas and carrots and simmer for about 1 minute.
-s

In [8]:
gpt2.generate(sess,
              run_name='run0',
              length=250,
              temperature=1.0,
              prefix="<|startoftext|><soi>chicken,orange,ginger,",
              truncate="<|endoftext|>",
              checkpoint_dir=checkpoint,
              nsamples=5,
              batch_size=5
              )

<|startoftext|><soi>chicken,orange,ginger,cumin seed,coriander seed,dried chileprune,lime in juice,onion,garlic clove,parsley,salt,black pepper,turmeric,ground tortilla,cheddar cheese<eoi>
<sot>spicy cumin or yellow tang chicken spaghetti<eot>
<soc>-to make the soup. cut the onion using 1 mandoline or knife to use for cooking .
-breakred chillies, ginger, cumin and coriander seeds, dice or dice in 1/2 and put to one side.
-grate the cloves and place warm coriander to one side.
-prepare the onion, ground coriander, garlic and other spices by grinding coast to bottom.
-put them into 1 sieve over 1 bowl filled with cold water.
-tear the chicken into thin chunks.
-preheat the grill to gas or medium heat.
-fry, lightly oil or preheat 1 nonstick frying pan until the chicken is thoroughly cooked.
-remove from pan, reserve 1/2 the cooked juices and discard it.
-return the meat and onion to pan and
<|startoftext|><soi>chicken,orange,ginger,vegetable oil,salt and freshly ground black pepper,gree